## Importar Librerias

In [ ]:
from psycopg2 import connect, Error
import pandas as pd
import csv
from io import StringIO
import glob
import functools


## Conexion

In [ ]:
def getConnection():
    try:
        connection = connect(host='localhost',database='amazon',user='postgres', password='1234', port='5432')   
    except(Exception, Error) as error:
        connection.rollback()
        print("Error: %s" % error)
    return connection

## Inserts query

In [ ]:
def insert_query(query,data):
    try:
        con = getConnection()
        if con and query != '':
            cursor = con.cursor()
            cursor.execute(query,data)
        con.commit()
    except(Exception, Error) as error:
        print("Error: %s" % error)

## Cargar product_category

In [ ]:
files = glob.glob('*.tsv')
query = "insert into category(category_name) values (%s)"

for file in files:
    start = len("amazon_reviews_us_")
    end = file.index("_v1_00.tsv")
    name = file[start:end]
    insert_query(query, (name,))


## Crear output para cargar en la base de datos

In [ ]:
def to_database(dataframe,table):
    output = StringIO()
    dataframe.to_csv(output, sep='\t', index = False, header = False, quoting=csv.QUOTE_NONE, escapechar='\\')
    output.seek(0)
    connection = getConnection()
    cursor = connection.cursor()
    cursor.copy_from(output,table,null='')
    connection.commit()
    cursor.close()

## Cargar Customer

In [ ]:
df_customers_id = pd.concat(map(functools.partial(pd.read_csv, sep='\t',on_bad_lines='skip',usecols=[1]),glob.glob('*.tsv')))
df_customers_id = df_customers_id.drop_duplicates(subset="customer_id")

to_database(df_customers_id,"customer")


## Cargar Product

In [ ]:
df_products = pd.concat(map(functools.partial(pd.read_csv, sep='\t',on_bad_lines='skip',usecols=[3,4,5,6]),glob.glob('*.tsv')))
df_products = df_products.drop_duplicates(subset="product_id")

to_database(df_products,"product")

## Cargar Review

In [ ]:
#Se leen todos los archivos que se encuentren en la carpeta terminados en .tsv

df_products = pd.concat(map(functools.partial(pd.read_csv, sep='\t',on_bad_lines='skip',usecols=[2,7,8,9,10,11,12,13,14]),glob.glob('*.tsv')))

